In [3]:
import os
from dotenv import load_dotenv
from prettytable import PrettyTable
from pathlib import Path
import google.generativeai as genai

/Users/sameeraboppana/work_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
load_dotenv()

API_KEY = os.getenv("API_KEY")

In [6]:
# Function to configure the API key
def configure_api_key(api_key):
    genai.configure(api_key=api_key)
    return {"api_key": api_key}

# Function to configure the generative model
def configure_model(config, safety_settings):
    model = genai.GenerativeModel(
        model_name="gemini-pro-vision",
        generation_config=config,
        safety_settings=safety_settings
    )
    return {"model": model}

# Function to validate and read the image
def load_image(image_path):
    path = Path(image_path)
    if not path.exists():
        raise FileNotFoundError(f"Could not find image: {path}")
    image_data = path.read_bytes()
    return {"image_data": image_data, "mime_type": "image/webp"}

# Function to read the prompt content
def read_prompt(prompt_file):
    with open(prompt_file, "r", encoding="utf-8", errors="ignore") as file:
        content = file.read()
    return {"prompt_content": content}

# Function to manage the prompt
def manage_prompt(prompt_content, additional_context=""):
    if additional_context:
        prompt_content += "\n" + additional_context
    return {"managed_prompt": prompt_content}

# Function to generate content using the model
def generate_content(model, image_data, managed_prompt):
    prompt_parts = [{"mime_type": "image/webp", "data": image_data}, managed_prompt]
    response = model.generate_content(prompt_parts)
    return {"response_text": response.text}

# Function to perform safety and compliance check
def check_safety_compliance(response_text):
    if "inappropriate_keyword" in response_text:
        raise ValueError("Generated content does not meet safety standards.")
    return {"safe_response_text": response_text}

# Function to handle the response
def handle_response(response_text):
    parsed_response = response_text  # This can be more complex depending on requirements
    return {"parsed_response": parsed_response}

# Function to chain the functions
def run_pipeline(api_key, image_path, prompt_file, additional_context, config, safety_settings):
    data = {}
    data.update(configure_api_key(api_key))
    data.update(configure_model(config, safety_settings))
    data.update(load_image(image_path))
    data.update(read_prompt(prompt_file))
    managed_prompt = manage_prompt(data["prompt_content"], additional_context)
    data.update({"managed_prompt": managed_prompt["managed_prompt"]})
    generated_content = generate_content(data["model"], data["image_data"], data["managed_prompt"])
    data.update({"response_text": generated_content["response_text"]})
    safe_content = check_safety_compliance(data["response_text"])
    data.update({"safe_response_text": safe_content["safe_response_text"]})
    parsed_response = handle_response(data["safe_response_text"])
    data.update({"parsed_response": parsed_response["parsed_response"]})
    return data

# Define a function to run the pipeline with the option to add more prompts
def run_pipeline_with_additional_prompt(api_key, image_path, prompt_file,  config, safety_settings, additional_context =""):
    final_response = {}
    while True:
        # Run the initial pipeline
        result = run_pipeline(api_key, image_path, prompt_file, additional_context, config, safety_settings)

        # Display the response
        print("Response:")
        print(result["parsed_response"])

        # Ask the user if they want to add more prompts
        choice = input("Do you want to add another prompt? (yes/no): ").lower()
        if choice != 'yes':
            final_response[prompt_file] = result["parsed_response"]
            break

        # Prompt the user for additional input
        additional_context = input("Enter additional prompt or context: ")

    return final_response


# Function to write the final response table to a file
def write_response_to_file(final_response, file_path):
    with open(file_path, 'w') as file:
        file.write("# Final Response Table\n\n")
        for section, content in final_response.items():
            file.write(f"## {section}\n")
            if isinstance(content, list):
                for i, item in enumerate(content, start=1):
                    if len(content) > 1:
                        file.write(f"{i}. **{item}**\n")  # Bold if multiple items in a section
                    else:
                        file.write(f"{i}. {item}\n")
            else:
                file.write(f"{content}\n")
            file.write("\n")
    print(f"Final response table written to {file_path}")



# Define the file path where you want to write the table
file_path = "final_response_table.txt"



# Define the configuration parameters
api_key = API_KEY
image_path = "./ON31.webp"
prompt_file = "./prompts/geographic.txt"
generation_config = {
    "temperature": 0.4,
    "top_p": 1,
    "top_k": 32,
    "max_output_tokens": 4096,
}
safety_settings = [
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_MEDIUM_AND_ABOVE"},
]

# Call the function to run the pipeline with the option to add more prompts
final_response = run_pipeline_with_additional_prompt(api_key, image_path, prompt_file, generation_config, safety_settings)

# Write the final response table to the file
write_response_to_file(final_response, file_path)


Response:
 - **Countries with the highest interest:**
 - Canada
 - United States
 - Northern Europe (Sweden, Norway, Finland, Denmark)
- **Cultural insights:**
 - The sweater's design is reminiscent of traditional Nordic sweaters, which are popular in countries with cold climates.
 - The colors red and white are often associated with Christmas and the holiday season, which makes the sweater a popular choice for festive occasions.
 - The sweater's Fair Isle pattern is a traditional knitting technique that is often used in Nordic sweaters.
 - **Marketing messages:**
 - "Stay warm and stylish this holiday season with our new Nordic-inspired sweater."
 - "Get cozy with our Fair Isle sweater, perfect for the holidays."
 - "Be the hit of the party in our festive red and white sweater."
- **Culturally sensitive marketing:**
 - Avoid using images of people from cultures other than the ones you are targeting.
 - Be aware of the local customs and traditions surrounding the holiday season.
 - Use